In [64]:
#A brute force implementation of Algorithm1

In [168]:
import pandas as pd
import numpy as np
from random import sample,seed

In [169]:
LAMB = 0.04

In [170]:
df = pd.DataFrame(pd.read_csv('../data/compas-binary.csv'))
df

seed(10)

idx = sample(range(df.shape[0]),df.shape[0])

In [171]:
x = df.as_matrix()[:,:13]

y = df.as_matrix()[:,13]

In [172]:
# calculate the bound with dp=all leaves except leaf l
def bound(tree, x, y, l, lamb = LAMB):
    return (sum(tree.get_cap()[:l]+tree.get_cap()[l+1:])-
            sum(tree.get_ncc()[:l]+tree.get_ncc()[l+1:]))/len(y) + lamb*len(tree.get_prefix())

# calculate the risk
def Risk(tree, x, y, lamb = LAMB):
    return 1-sum(tree.get_ncc())/len(y) + + lamb*len(tree.get_prefix())

In [173]:
def calcul(prefix, x, y):
    """
    Function for calculating the predictions, number of data captured,
    and number of data correctly captured by the leaves.
    """
    prediction = []
    num_captured = []
    num_captured_incorrect = []
    for i in range(len(prefix)):
        tag = np.array([True]*len(y))
        for j in range(len(prefix[i])):
            rule_index = abs(prefix[i][j])-1
            rule_label = int(prefix[i][j]>0)
            #print("prefix:",prefix)
            #print("rule_index:",rule_index)
            #print("x:",x)
            tag = (x[:,rule_index] == rule_label)*tag
            
        # the y's of these data captured by leaf i
        y_leaf = y[tag]
        
        num_cap = len(y_leaf)
        num_captured.append(num_cap)
        
        if len(y_leaf)>0:
            pred = int(y_leaf.sum()/len(y_leaf) > 0.5)
            prediction.append(pred)
            num_cap_incor = sum(y_leaf != pred)
            num_captured_incorrect.append(num_cap_incor)
        else:
            prediction.append(0)
            num_captured_incorrect.append(0)
        
    return prediction, num_captured, num_captured_incorrect

In [174]:
aaaa = [[1,0,0],
        [0,1,1]]
aaaa.append([1,1,1])
aaaa[1:]

[[0, 1, 1], [1, 1, 1]]

In [175]:
class CacheTree:
    """
    A tree data structure.
    prefix: a 2-d tuple to record the prefixes of leaves
    prediction: a list to record the predictions of leaves
    num_captured: a list to record number of data captured by the leaves
    num_captured_correct: a list to record number of data correctly captured by the leaves
    """
    def __init__(self, x, y, prefix, 
                 prediction=None,
                 num_captured=None,
                 num_captured_incorrect=None,
                 deadleaf = None,
                 splitleaf = None,
                 lbound=None,lamb=LAMB,tag = False):
        self.prefix = prefix
        self.prediction = prediction
        self.num_captured = num_captured
        self.num_captured_incorrect = num_captured_incorrect
        #self.split = split #which leaf to split
        self.deadleaf = deadleaf #a list indicate which leaves will never be split (support bound)
        self.splitleaf = splitleaf #a list indicate which leaves will be split
        self.lbound = lbound #a list of lower bound
        self.tag = tag
        
        if prediction==None:
            l = len(prefix)
            self.prediction, self.num_captured, self.num_captured_incorrect = calcul(self.prefix, x, y)
            self.deadleaf = [0]*l
            self.splitleaf = [[1]*l]
            self.lbound = [sum(self.num_captured_incorrect[:i]+self.num_captured_incorrect[i+1:])/len(y) + lamb*len(self.prefix) 
                           for i in range(l)]
            
    def get_prefix(self):
        return self.prefix
    
    def get_pred(self):
        return self.prediction
    
    def get_cap(self):
        return self.num_captured
    
    def get_ncc(self):
        return self.num_captured_incorrect
    
    def get_deadleaf(self):
        return self.deadleaf
    
    def get_splitleaf(self):
        return self.splitleaf
    
    def set_deadleaf(self,i):
        self.deadleaf[i] = 1
        return
    
    def set_not_splitleaf(self,i):
        self.splitleaf[i] = 0
        return
        
    def get_lbound(self):
        return self.lbound
    
    def get_tag(self):
        return self.tag

In [176]:
[i**2
 for i in range(3)]

[0, 1, 4]

In [177]:
from gmpy2 import mpz
from gmpy2 import mpfr
import gmpy2

In [178]:
gmpy2.get_context().precision=100

In [179]:
aaa=mpfr('2')
bbb=mpfr('0.3')

In [180]:
aaa

mpfr('2.0',100)

In [181]:
bbb

mpfr('0.30000000000000000000000000000016',100)

In [182]:
aaa+bbb

mpfr('2.2999999999999999999999999999994',100)

In [183]:
mpz(2)/mpz(3)

mpfr('0.66666666666666666666666666666693',100)

In [184]:
class Eliminate:
    """
    A data structure to record and identify
    whether a tree has been visited/pruned
    """
    def __init__(self, elim_dict = None, 
                 eliminated = None):
        self.elim_dict = {}
        self.eliminated = []
        
    def eliminate(self, prefix):
        self.elim_dict[prefix] = 1
    
    def eliminate_children(self, dp):
        self.eliminated.append(dp)
        
    def is_eliminated(self, prefix):
        if prefix in self.elim_dict.keys():
            #print("Eliminated!")
            return True
        
        for i in range(len(self.eliminated)):
            if all([r in prefix for r in self.eliminated[i]]):
                #print("Eliminated!")
                return True
        
        return False

In [185]:
import time

In [186]:
'''def algorithm1_multi(x, y, lamb = LAMB):
    """
    A brute forece implementation of Algorithm1
    ## multiple copies of tree
    ## mark which leaf to split
    """
    
    d_c = None
    R_c = 1

    nrule = x.shape[1]
    ndata = len(y)
    print("ndata:", ndata)

    # initialize the queue to include all trees of just one split
    queue = [CacheTree(prefix = ((-r,),(r,)), x = x, y = y) for r in range(1, nrule+1)]

    E = Eliminate()

    while (queue):
        tree = queue.pop(0)
        d = tree.get_prefix()
        if E.is_eliminated(d):
            continue
        else:
            E.eliminate(d)
        R = Risk(tree,x,y,lamb)
        if R<R_c:
            d_c = d
            R_c = R

        #print("d",d)
        #print("R",R)
        #print("R_c",R_c)
        #print(tree.get_ncc())
        #print(tree.get_cap())
        
        #for i in range(len(d)):
        i = tree.get_split()
            
        #(Lower bound on antecedent support)
        if tree.get_cap()[i]/ndata/2 < lamb:
            continue
            
        d0 = d[i]
        dp = d[:i]+d[i+1:]
            #print("bound", bound(tree,x,y,i))
            #print("R_c", R_c)
            
            
        if bound(tree,x,y,i,lamb)<R_c:
            for j in range(1, nrule+1):
                if (j not in d0)and(-j not in d0):
                    l1 = d0+(-j,)
                    l2 = d0+(j,)
                    t = dp+(l1, l2)
                    #print("t",t)
                    pred_l, cap_l, corr_l = calcul((l1,l2),x,y)

                    prediction = tree.get_pred()[:i]+tree.get_pred()[i+1:]+pred_l,
                    num_captured = tree.get_cap()[:i]+tree.get_cap()[i+1:]+cap_l,
                    num_captured_correct = tree.get_ncc()[:i]+tree.get_ncc()[i+1:]+corr_l,

                    for sp in range(len(t)):
                        tree_new = CacheTree(prefix = t,
                                             prediction = prediction,
                                             num_captured = num_captured,
                                             num_captured_correct = num_captured_correct,
                                             lbound = None,
                                             split = spl)
                        if (not E.is_eliminated(t)):
                            queue.append(tree_new)
                    
                    #(Lower bound on accurate antecedent support)
                    #a_l = sum(corr_l)/ndata - sum(cap_l)/ndata/2
                    #if a_l < lamb:
                    #    bound_theorem11(l1,l2,t,tree_new,E,queue,nrule,x,y)
                        #    E.eliminate_children((l1,l2))
                        ########### split l1/l2??????????????????????????????????????
                    #elif (not E.is_eliminated(t)):
                    #    #pred_l, cap_l, corr_l = calcul((l1,l2),x,y)
                    #    queue.append(tree_new)
                        
        else:
            E.eliminate_children(dp)
            for j in range(1, nrule+1):
                if (j not in d0)and(-j not in d0):
                    l1 = d0+(-j,)
                    l2 = d0+(j,)
                    t = dp+(l1, l2)
                    #print("t",t)
                    pred_l, cap_l, corr_l = calcul((l1,l2),x,y)

                    prediction = tree.get_pred()[:i]+tree.get_pred()[i+1:]+pred_l,
                    num_captured = tree.get_cap()[:i]+tree.get_cap()[i+1:]+cap_l,
                    num_captured_correct = tree.get_ncc()[:i]+tree.get_ncc()[i+1:]+corr_l,

                    for sp in range(len(t)-2):
                        tree_new = CacheTree(prefix = t,
                                             prediction = prediction,
                                             num_captured = num_captured,
                                             num_captured_correct = num_captured_correct,
                                             lbound = None,
                                             split = spl)
                        if (not E.is_eliminated(t)):
                            queue.append(tree_new)
            

    print(d_c)
    print("R_c", R_c)'''

'def algorithm1_multi(x, y, lamb = LAMB):\n    """\n    A brute forece implementation of Algorithm1\n    ## multiple copies of tree\n    ## mark which leaf to split\n    """\n    \n    d_c = None\n    R_c = 1\n\n    nrule = x.shape[1]\n    ndata = len(y)\n    print("ndata:", ndata)\n\n    # initialize the queue to include all trees of just one split\n    queue = [CacheTree(prefix = ((-r,),(r,)), x = x, y = y) for r in range(1, nrule+1)]\n\n    E = Eliminate()\n\n    while (queue):\n        tree = queue.pop(0)\n        d = tree.get_prefix()\n        if E.is_eliminated(d):\n            continue\n        else:\n            E.eliminate(d)\n        R = Risk(tree,x,y,lamb)\n        if R<R_c:\n            d_c = d\n            R_c = R\n\n        #print("d",d)\n        #print("R",R)\n        #print("R_c",R_c)\n        #print(tree.get_ncc())\n        #print(tree.get_cap())\n        \n        #for i in range(len(d)):\n        i = tree.get_split()\n            \n        #(Lower bound on antecedent

In [210]:
[[1,2],[1,3]]+[[1,2]]

[[1, 2], [1, 3], [1, 2]]

In [230]:
def algorithm1_multi(x, y, lamb = LAMB):
    """
    A brute forece implementation of Algorithm1
    ## multiple copies of tree
    ## mark which leaf to split
    """
    
    d_c = None
    R_c = 1

    nrule = x.shape[1]
    ndata = len(y)
    print("ndata:", ndata)

    # initialize the queue to include all trees of just one split
    queue = [CacheTree(prefix = ((-r,),(r,)), x = x, y = y) for r in range(1, nrule+1)]

    E = Eliminate()

    while (queue):
        tree = queue.pop(0)
        d = tree.get_prefix()
        if E.is_eliminated(d):
            continue
        else:
            E.eliminate(d)
        R = tree.get_lbound()[0]+(tree.get_ncc()[0])/len(y)#Risk(tree,x,y,lamb)
        if R<R_c:
            d_c = d
            R_c = R

        print("d",d)
        print("R",R)
        print("R_c",R_c)
        #print(tree.get_ncc())
        #print(tree.get_cap())
        #print(tree.get_lbound())
        
        for i in range(len(d)):
            if tree.get_deadleaf()[i]==1:
                continue
            
            #(Lower bound on antecedent support)
            if tree.get_cap()[i]/ndata/2 < lamb:
                tree.set_deadleaf(i)
                continue
            
            spl = tree.get_splitleaf()
            split_next = spl[1:]
            
            if spl[0][i]==0:
                continue
            
            print("splitleaf():",spl)
            #if len(split_next)>0:
            #    print("split_next[k][:i]",split_next[0][:i])
            #    print("split_next[k][i]",split_next[0][i])
            #    print("split_next[k][i+1:]",split_next[0][i+1:])
            splitleaf_list = [split_next[k][:i]+split_next[k][i:i+1]*2+split_next[k][i+1:] 
                              for k in range(len(split_next))]
            
            d0 = d[i]
            dp = d[:i]+d[i+1:]
            #print("bound", bound(tree,x,y,i))
            #print("R_c", R_c)
            
            
            #if bound(tree,x,y,i,lamb)<R_c:
            if tree.get_lbound()[i]<R_c:
                print("===CASE 1===")
                for j in range(1, nrule+1):
                    if (j not in d0)and(-j not in d0):
                        l1 = d0+(-j,)
                        l2 = d0+(j,)
                        t = dp+(l1, l2)
                        #print("t",t)
                        pred_l, cap_l, incorr_l = calcul((l1,l2),x,y)
                        loss_l1 = (incorr_l[0])/len(y)
                        loss_l2 = (incorr_l[1])/len(y)
                        loss_d0 = tree.get_ncc()[i]/len(y)
                        delta = loss_l1+loss_l2-loss_d0+lamb
                        old_lbound = tree.get_lbound()[:i]+tree.get_lbound()[i+1:]
                        new_lbound = [b+delta for b in old_lbound]+[tree.get_lbound()[i]+loss_l2+lamb,tree.get_lbound()[i]+loss_l1+lamb]
                        
                        splitleaf = [1]*len(t)
                        
                        #(Lower bound on accurate antecedent support)
                        a_l = (sum(cap_l)-sum(incorr_l))/ndata - sum(cap_l)/ndata/2
                        if a_l < lamb:
                            print("===CASE 2===")
                            splitleaf = [0]*(len(t)-2)+[1,1] #next round split l1/l2
                            #bound_theorem11(l1,l2,t,tree_new,E,queue,nrule,x,y)
                        #    E.eliminate_children((l1,l2))
                            ########### split l1/l2??????????????????????????????????????
                        
                        #if tree.get_tag()==True:
                        #    coma = [co==0 for co in tree.get_comaleaf()]
                        #    comaleaf = coma[:i]+coma[i+1:]+[1,1]
                        
                        sl = splitleaf_list+[splitleaf]
                        #print("here", sl)
                        tree_new = CacheTree(x = x, y = y, prefix = t,
                                             prediction = tree.get_pred()[:i]+tree.get_pred()[i+1:]+pred_l,
                                             num_captured = tree.get_cap()[:i]+tree.get_cap()[i+1:]+cap_l,
                                             num_captured_incorrect = tree.get_ncc()[:i]+tree.get_ncc()[i+1:]+incorr_l,
                                             deadleaf = tree.get_deadleaf()[:i]+tree.get_deadleaf()[i+1:]+[0,0],
                                             splitleaf = sl,
                                             lbound = new_lbound)
                        
                        if (not E.is_eliminated(t)):
                            #pred_l, cap_l, corr_l = calcul((l1,l2),x,y)
                            queue.append(tree_new)
                        
            else:
                print("===CASE 3===")
                #E.eliminate_children(dp)
                for j in range(1, nrule+1):
                    if (j not in d0)and(-j not in d0):
                        l1 = d0+(-j,)
                        l2 = d0+(j,)
                        t = dp+(l1, l2)
                        #print("t",t)
                        pred_l, cap_l, incorr_l = calcul((l1,l2),x,y)
                        loss_l1 = (incorr_l[0])/len(y)
                        loss_l2 = (incorr_l[1])/len(y)
                        loss_d0 = tree.get_ncc()[i]/len(y)
                        delta = loss_l1+loss_l2-loss_d0+lamb
                        old_lbound = tree.get_lbound()[:i]+tree.get_lbound()[i+1:]
                        new_lbound = [b+delta for b in old_lbound]+[tree.get_lbound()[i]+loss_l2+lamb,tree.get_lbound()[i]+loss_l1+lamb]
                        
                        splitleaf = [1]*(len(t)-2)+[0,0]
                        
                        sl = splitleaf_list+[splitleaf]
                        
                        #(Lower bound on accurate antecedent support)
                        a_l = (sum(cap_l)-sum(incorr_l))/ndata - sum(cap_l)/ndata/2
                        if a_l < lamb:
                            print("===CASE 4===")
                            splitleaf = [0]*(len(t)-2)+[1,1] #next round split l1/l2
                            sl = sl+[splitleaf]
                            #bound_theorem11(l1,l2,t,tree_new,E,queue,nrule,x,y)
                        #    E.eliminate_children((l1,l2))
                            ########### split l1/l2??????????????????????????????????????
                        
                        #if tree.get_tag()==True:
                        #    coma = [co==0 for co in tree.get_comaleaf()]
                        #    comaleaf = coma[:i]+coma[i+1:]+[1,1]
                        
                        print("sl:",sl)
                        tree_new = CacheTree(x = x, y = y, prefix = t,
                                             prediction = tree.get_pred()[:i]+tree.get_pred()[i+1:]+pred_l,
                                             num_captured = tree.get_cap()[:i]+tree.get_cap()[i+1:]+cap_l,
                                             num_captured_incorrect = tree.get_ncc()[:i]+tree.get_ncc()[i+1:]+incorr_l,
                                             deadleaf = tree.get_deadleaf()[:i]+tree.get_deadleaf()[i+1:]+[0,0],
                                             splitleaf = sl,
                                             lbound = new_lbound)
                        print("tree_new:",tree_new.get_prefix())
                        print("tree_new:",tree_new.get_splitleaf())
                        
                        if (not E.is_eliminated(t)):
                            #pred_l, cap_l, corr_l = calcul((l1,l2),x,y)
                            queue.append(tree_new)
                #print("Children Elim")

    print(d_c)
    print("R_c", R_c)

#### for 3 rules, 25 data, lambda = 0.04
algorithm  algorithm1  algorithm1_2splits  algorithm1_nobound
time     20.7ms    51.9ms          217ms

In [188]:
aaaaaa = ((1,2),(3,4))


In [189]:
aaaaaa.index((1,2))

0

In [190]:
rule_idx = sample(range(13),13)

In [191]:
%%time
# algorithm1_2splits, 3 rules, 25 data, lambda = 0.04

algorithm1_2splits(x[:,rule_idx[:5]][idx[:25]],y[idx[:25]],lamb=0.04)

NameError: name 'algorithm1_2splits' is not defined

In [229]:
%%time
# algorithm1_2splits, 3 rules, 25 data, lambda = 0.04

algorithm1_multi(x[:,rule_idx[:4]][idx[:100]],y[idx[:100]])

ndata: 100
d ((-1,), (1,))
R 0.49
R_c 0.49
splitleaf(): [[1, 1]]
===CASE 1===
splitleaf(): [[1, 1]]
===CASE 1===
===CASE 2===
===CASE 2===
d ((-2,), (2,))
R 0.48
R_c 0.48
splitleaf(): [[1, 1]]
===CASE 1===
splitleaf(): [[1, 1]]
===CASE 1===
===CASE 2===
===CASE 2===
===CASE 2===
d ((-3,), (3,))
R 0.45
R_c 0.45
splitleaf(): [[1, 1]]
===CASE 1===
splitleaf(): [[1, 1]]
===CASE 1===
===CASE 2===
===CASE 2===
===CASE 2===
d ((-4,), (4,))
R 0.49000000000000005
R_c 0.45
splitleaf(): [[1, 1]]
===CASE 1===
d ((1,), (-1, -2), (-1, 2))
R 0.52
R_c 0.45
splitleaf(): [[1, 1, 1]]
===CASE 1===
===CASE 2===
===CASE 2===
splitleaf(): [[1, 1, 1]]
===CASE 1===
splitleaf(): [[1, 1, 1]]
===CASE 1===
===CASE 2===
===CASE 2===
d ((1,), (-1, -3), (-1, 3))
R 0.51
R_c 0.45
splitleaf(): [[1, 1, 1]]
===CASE 1===
===CASE 2===
===CASE 2===
splitleaf(): [[1, 1, 1]]
===CASE 1===
splitleaf(): [[1, 1, 1]]
===CASE 3===
===CASE 4===
sl: [[1, 1, 0, 0], [0, 0, 1, 1]]
tree_new: ((1,), (-1, -3), (-1, 3, -2), (-1, 3, 2))
tree_

tree_new: [[1, 1, 1, 0, 0]]
d ((1,), (-1, 3), (-1, -3, -2), (-1, -3, 2))
R 0.53
R_c 0.45
splitleaf(): [[1, 1, 1, 1]]
===CASE 3===
===CASE 4===
sl: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
tree_new: ((-1, 3), (-1, -3, -2), (-1, -3, 2), (1, -2), (1, 2))
tree_new: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
sl: [[1, 1, 1, 0, 0]]
tree_new: ((-1, 3), (-1, -3, -2), (-1, -3, 2), (1, -3), (1, 3))
tree_new: [[1, 1, 1, 0, 0]]
===CASE 4===
sl: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
tree_new: ((-1, 3), (-1, -3, -2), (-1, -3, 2), (1, -4), (1, 4))
tree_new: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
splitleaf(): [[1, 1, 1, 1]]
===CASE 3===
===CASE 4===
sl: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
tree_new: ((1,), (-1, -3, -2), (-1, -3, 2), (-1, 3, -2), (-1, 3, 2))
tree_new: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
===CASE 4===
sl: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
tree_new: ((1,), (-1, -3, -2), (-1, -3, 2), (-1, 3, -4), (-1, 3, 4))
tree_new: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
splitleaf(): [[1, 1, 1, 1]]
===CASE 1===
splitleaf(): [[1, 1,

tree_new: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
===CASE 4===
sl: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
tree_new: ((-3, 1), (-3, -1, -4), (-3, -1, 4), (3, -2), (3, 2))
tree_new: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
===CASE 4===
sl: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
tree_new: ((-3, 1), (-3, -1, -4), (-3, -1, 4), (3, -4), (3, 4))
tree_new: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
splitleaf(): [[1, 1, 1, 1]]
===CASE 3===
sl: [[1, 1, 1, 0, 0]]
tree_new: ((3,), (-3, -1, -4), (-3, -1, 4), (-3, 1, -2), (-3, 1, 2))
tree_new: [[1, 1, 1, 0, 0]]
sl: [[1, 1, 1, 0, 0]]
tree_new: ((3,), (-3, -1, -4), (-3, -1, 4), (-3, 1, -4), (-3, 1, 4))
tree_new: [[1, 1, 1, 0, 0]]
splitleaf(): [[1, 1, 1, 1]]
===CASE 1===
d ((3,), (-3, -1), (-3, 1, -2), (-3, 1, 2))
R 0.53
R_c 0.45
splitleaf(): [[1, 1, 1, 1]]
===CASE 3===
===CASE 4===
sl: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
tree_new: ((-3, -1), (-3, 1, -2), (-3, 1, 2), (3, -1), (3, 1))
tree_new: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]
===CASE 4===
sl: [[1, 1, 1, 0, 0], [0, 0, 0, 1, 1]]


tree_new: ((1,), (-1, 3), (-1, -3, -2, -4), (-1, -3, -2, 4), (-1, -3, 2, -4), (-1, -3, 2, 4))
tree_new: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
splitleaf(): [[1, 1, 1, 1, 1]]
===CASE 1===
d ((1,), (-1, 3), (-1, -3, -2), (-1, -3, 2, -4), (-1, -3, 2, 4))
R 0.57
R_c 0.45
splitleaf(): [[0, 0, 0, 1, 1]]
===CASE 3===
d ((1,), (-1, 3), (-1, -3, -4), (-1, -3, 4, -2), (-1, -3, 4, 2))
R 0.57
R_c 0.45
splitleaf(): [[1, 1, 1, 1, 1]]
===CASE 3===
===CASE 4===
sl: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
tree_new: ((-1, 3), (-1, -3, -4), (-1, -3, 4, -2), (-1, -3, 4, 2), (1, -2), (1, 2))
tree_new: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
sl: [[1, 1, 1, 1, 0, 0]]
tree_new: ((-1, 3), (-1, -3, -4), (-1, -3, 4, -2), (-1, -3, 4, 2), (1, -3), (1, 3))
tree_new: [[1, 1, 1, 1, 0, 0]]
===CASE 4===
sl: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
tree_new: ((-1, 3), (-1, -3, -4), (-1, -3, 4, -2), (-1, -3, 4, 2), (1, -4), (1, 4))
tree_new: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
splitleaf(): [[1, 1, 1, 1, 1]]
===C

sl: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
tree_new: ((-2, -4), (-2, 4, 3), (-2, 4, -3, -1), (-2, 4, -3, 1), (2, -4), (2, 4))
tree_new: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
splitleaf(): [[1, 1, 1, 1, 1]]
===CASE 3===
===CASE 4===
sl: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
tree_new: ((2,), (-2, -4), (-2, 4, -3, -1), (-2, 4, -3, 1), (-2, 4, 3, -1), (-2, 4, 3, 1))
tree_new: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
splitleaf(): [[1, 1, 1, 1, 1]]
===CASE 1===
splitleaf(): [[1, 1, 1, 1, 1]]
===CASE 3===
d ((3,), (-3, -1, -2), (-3, -1, 2), (-3, 1, -2), (-3, 1, 2))
R 0.55
R_c 0.45
splitleaf(): [[1, 1, 1, 1, 1]]
===CASE 3===
===CASE 4===
sl: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
tree_new: ((-3, -1, -2), (-3, -1, 2), (-3, 1, -2), (-3, 1, 2), (3, -1), (3, 1))
tree_new: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
===CASE 4===
sl: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
tree_new: ((-3, -1, -2), (-3, -1, 2), (-3, 1, -2), (-3, 1, 2), (3, -2), (3, 2))
tree_new: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 

tree_new: [[1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1]]
splitleaf(): [[1, 1, 1, 1, 1]]
===CASE 1===
splitleaf(): [[1, 1, 1, 1, 1]]
===CASE 3===
d ((-4,), (4, 3), (4, -3, -2), (4, -3, 2, -1), (4, -3, 2, 1))
R 0.56
R_c 0.45
splitleaf(): [[0, 0, 0, 1, 1]]
===CASE 3===
d ((-2, -3), (-2, 3), (2, 1), (2, -1, 3), (2, -1, -3, -4), (2, -1, -3, 4))
R 0.58
R_c 0.45
splitleaf(): [[0, 0, 0, 0, 1, 1]]
===CASE 3===
d ((-2, -3), (-2, 3), (2, 1), (2, -1, -4), (2, -1, 4, -3), (2, -1, 4, 3))
R 0.58
R_c 0.45
splitleaf(): [[0, 0, 0, 0, 1, 1]]
===CASE 3===
d ((-2, -3), (-2, 3), (2, 3), (2, -3, 1), (2, -3, -1, -4), (2, -3, -1, 4))
R 0.58
R_c 0.45
splitleaf(): [[0, 0, 0, 0, 1, 1]]
===CASE 3===
d ((-2, -3), (-2, 3), (2, 3), (2, -3, -4), (2, -3, 4, -1), (2, -3, 4, 1))
R 0.58
R_c 0.45
splitleaf(): [[0, 0, 0, 0, 1, 1]]
===CASE 3===
d ((-2, -3), (-2, 3), (2, -4), (2, 4, 1), (2, 4, -1, -3), (2, 4, -1, 3))
R 0.58
R_c 0.45
splitleaf(): [[0, 0, 0, 0, 1, 1]]
===CASE 3===
d ((-2, -3), (-2, 3), (2, -4), (2, 4, 3), (2, 4, -3, 

In [270]:
%%time
# algorithm1_2splits, 3 rules, 25 data, lambda = 0.04

algorithm1_2splits(x[:,rule_idx[:4]],y,lamb=0.04)

ndata: 6907
((-4,), (4,))
R_c 0.48495149847980307
CPU times: user 31.1 s, sys: 8 ms, total: 31.1 s
Wall time: 31.2 s


In [49]:
%%time
# 3 rules, 25 data, lambda = 0.04

algorithm1(x[:,rule_idx[:3]][idx[:25]],y[idx[:25]],lamb=0.04)

((-1,), (1,))
R_c 0.48000000000000004
CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 20.7 ms


In [14]:
%%time
# 4 rules, 25 data, lambda = 0.04

algorithm1(x[:,rule_idx[:4]][idx[:25]],y[idx[:25]],lamb=0.04)

((-1,), (1,))
R_c 0.48000000000000004
CPU times: user 392 ms, sys: 0 ns, total: 392 ms
Wall time: 394 ms


In [15]:
%%time
# 5 rules, 25 data, lambda = 0.04

algorithm1(x[:,rule_idx[:5]][idx[:25]],y[idx[:25]],lamb=0.04)

((-1,), (1,))
R_c 0.48000000000000004
CPU times: user 4.64 s, sys: 0 ns, total: 4.64 s
Wall time: 4.64 s


In [16]:
%%time
# 6 rules, 25 data, lambda = 0.04

algorithm1(x[:,rule_idx[:6]][idx[:25]],y[idx[:25]],lamb=0.04)

((-6,), (6,))
R_c 0.36000000000000004
CPU times: user 12.4 s, sys: 12 ms, total: 12.4 s
Wall time: 12.5 s


In [17]:
%%time
# 7 rules, 25 data, lambda = 0.04

algorithm1(x[:,rule_idx[:7]][idx[:25]],y[idx[:25]],lamb=0.04)

((-6,), (6,))
R_c 0.36000000000000004
CPU times: user 2min 29s, sys: 36 ms, total: 2min 29s
Wall time: 2min 30s


In [18]:
%%time
# 8 rules, 25 data, lambda = 0.04

algorithm1(x[:,rule_idx[:8]][idx[:25]],y[idx[:25]],lamb=0.04)

((-6,), (6,))
R_c 0.36000000000000004
CPU times: user 22min 48s, sys: 544 ms, total: 22min 48s
Wall time: 22min 49s


In [19]:
%%time
# 9 rules, 25 data, lambda = 0.04

algorithm1(x[:,rule_idx[:9]][idx[:25]],y[idx[:25]],lamb=0.04)

KeyboardInterrupt: 

In [20]:
%%time
# all 13 rules, 25 data, lambda = 0.04

algorithm1(x[idx[:25]],y[idx[:25]],lamb=0.04)

KeyboardInterrupt: 

In [21]:
%%time
# 6 rules, 25 data, lambda = 0.03

algorithm1(x[:,rule_idx[:6]][idx[:25]],y[idx[:25]],lamb=0.03)

((-6,), (6,))
R_c 0.34
CPU times: user 15.2 s, sys: 4 ms, total: 15.2 s
Wall time: 15.2 s


In [22]:
%%time
# 6 rules, 25 data, lambda = 0.02

algorithm1(x[:,rule_idx[:6]][idx[:25]],y[idx[:25]],lamb=0.02)

((-5, -6), (-5, 6), (5, 1), (5, -1, -2), (5, -1, 2))
R_c 0.29999999999999993
CPU times: user 27 s, sys: 0 ns, total: 27 s
Wall time: 27 s


In [23]:
%%time
# 6 rules, 25 data, lambda = 0.01

algorithm1(x[:,rule_idx[:6]][idx[:25]],y[idx[:25]],lamb=0.01)

((-5, -6), (-5, 6), (5, 1), (5, -1, -2), (5, -1, 2))
R_c 0.24999999999999994
CPU times: user 31.4 s, sys: 20 ms, total: 31.4 s
Wall time: 31.4 s


In [16]:
def algorithm1_nobound(x, y, lamb):
    """
    A brute forece implementation of Algorithm1 without the bound of Theorem1
    """
    
    d_c = None
    R_c = 1

    nrule = x.shape[1]

    # initialize the queue to include all trees of just one split
    queue = [CacheTree(prefix = ((-r,),(r,)), x = x, y = y) for r in range(1, nrule+1)]

    E = Eliminate()

    while (queue):
        tree = queue.pop(0)
        d = tree.get_prefix()
        if E.is_eliminated(d):
            continue
        else:
            E.eliminate(d)
        R = Risk(tree,x,y,lamb)
        if R<R_c:
            d_c = d
            R_c = R

        #print("d",d)
        #print("R",R)
        #print("R_c",R_c)
        #print(tree.get_pred())

        for i in range(len(d)):
            d0 = d[i]
            dp = d[:i]+d[i+1:]
            #print("bound", bound(tree,x,y,i))
            #print("R_c", R_c)
            #if bound(tree,x,y,i,lamb)<R_c:
            for j in range(1, nrule+1):
                if (j not in d0)and(-j not in d0):
                    l1 = d0+(-j,)
                    l2 = d0+(j,)
                    t = dp+(l1, l2)
                    #print("t",t)
                    if (not E.is_eliminated(t)):
                        pred_l, cap_l, corr_l = calcul((l1,l2),x,y)
                        queue.append(
                            CacheTree(prefix = t,
                                      prediction = tree.get_pred()[:i]+tree.get_pred()[i+1:]+pred_l,
                                      num_captured = tree.get_cap()[:i]+tree.get_cap()[i+1:]+cap_l,
                                      num_captured_correct = tree.get_ncc()[:i]+tree.get_ncc()[i+1:]+corr_l)
                        )
            #else:
                #E.eliminate_children(dp)
                #print("Children Elim")

    print(d_c)
    print("R_c", R_c)

In [50]:
%%time
# algorithm1 without the bound, 3 rules, 25 data, lambda = 0.04

algorithm1_nobound(x[:,rule_idx[:3]][idx[:25]],y[idx[:25]],lamb=0.04)

((-1,), (1,))
R_c 0.48000000000000004
CPU times: user 208 ms, sys: 8 ms, total: 216 ms
Wall time: 217 ms


In [29]:
%%time
# algorithm1 without the bound, 4 rules, 25 data, lambda = 0.04

algorithm1_nobound(x[:,rule_idx[:4]][idx[:25]],y[idx[:25]],lamb=0.04)

KeyboardInterrupt: 